<a href="https://colab.research.google.com/github/srihith95/Sentiment-Classification/blob/main/distilbert_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install datasets transformers

In [ ]:
from datasets import load_dataset

In [ ]:
raw_dataset = load_dataset('imdb')
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
i small_train_set = raw_dataset['train'].shuffle(seed=42).select([i for i in range(4000)])
small_test_set = raw_dataset['test'].shuffle(seed=42).select([i for i in range(4000)])


In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
def pre_process(examples):
  return tokenizer(examples['text'], truncation=True)

tokenized_train = small_train_set.map(pre_process, batched=True)
tokenized_test = small_test_set.map(pre_process, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
  accu = load_metric('accuracy')
  f1_score = load_metric('f1')

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)

  accuracy = accu.compute(predictions=predictions, references=labels)['accuracy']
  f1 = f1_score.compute(predictions=predictions, references=labels)['f1']

  return {'accuracy': accuracy, 'f1': f1}

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='drive/My Drive/models',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy='epoch'
)

trainer = Trainer(
    tokenizer=tokenizer,
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.277600


TrainOutput(global_step=500, training_loss=0.2775871276855469, metrics={'train_runtime': 401.3934, 'train_samples_per_second': 19.931, 'train_steps_per_second': 1.246, 'total_flos': 1050180838513728.0, 'train_loss': 0.2775871276855469, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

<ipython-input-10-3135168671d5>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accu = load_metric('accuracy')


{'eval_loss': 0.25649183988571167,
 'eval_accuracy': 0.9135,
 'eval_f1': 0.9146521953626049,
 'eval_runtime': 74.1272,
 'eval_samples_per_second': 53.961,
 'eval_steps_per_second': 3.373,
 'epoch': 2.0}